# Задание 3.2 - сверточные нейронные сети (CNNs) в PyTorch

Это упражнение мы буде выполнять в Google Colab - https://colab.research.google.com/  
Google Colab позволяет запускать код в notebook в облаке Google, где можно воспользоваться бесплатным GPU!  

Авторы курса благодарят компанию Google и надеятся, что праздник не закончится.

Туториал по настройке Google Colab:  
https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d  
(Keras инсталлировать не нужно, наш notebook сам установит PyTorch)


In [1]:
# Intstall PyTorch and download data
!pip3 install torch torchvision

!wget -c http://ufldl.stanford.edu/housenumbers/train_32x32.mat http://ufldl.stanford.edu/housenumbers/test_32x32.mat

--2020-03-25 11:42:06--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat’

train_32x32.mat     100%[===================>] 173.61M  77.8MB/s    in 2.2s    

2020-03-25 11:42:14 (77.8 MB/s) - ‘train_32x32.mat’ saved [182040794/182040794]

--2020-03-25 11:42:14--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Reusing existing connection to ufldl.stanford.edu:80.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  98.2MB/s    in 0.6s    

2020-03-25 11:42:14 (98.2 MB/s) - ‘test_32x32.mat’ saved [64275384/64275384]

FINISHED --2020-03-25 11:42:14--
Total wall clock time: 7.9s
Downloaded: 2 files, 235M i

In [0]:
from collections import namedtuple

import matplotlib.pyplot as plt
import numpy as np
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
from torch.utils.data.sampler import SubsetRandomSampler

from torchvision import transforms

In [0]:
device = torch.device("cuda:0") # Let's make sure GPU is available!

# Загружаем данные

In [0]:
# First, lets load the dataset
data_train = dset.SVHN('./', 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[0.43,0.44,0.47],
                                               std=[0.20,0.20,0.20])                           
                       ])
                      )
data_test = dset.SVHN('./', split='test', transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[0.43,0.44,0.47],
                                               std=[0.20,0.20,0.20])                           
                       ]))

Разделяем данные на training и validation.

На всякий случай для подробностей - https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [0]:
batch_size = 64

data_size = data_train.data.shape[0]
validation_split = .2
split = int(np.floor(validation_split * data_size))
indices = list(range(data_size))
np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, 
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size,
                                         sampler=val_sampler)

In [0]:
# We'll use a special helper module to shape it into a flat tensor
class Flattener(nn.Module):
    def forward(self, x):
        batch_size, *_ = x.shape
        return x.view(batch_size, -1)

Создадим простейшую сеть с новыми слоями:  
Convolutional - `nn.Conv2d`  
MaxPool - `nn.MaxPool2d`

In [0]:
nn_model = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(4),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(4),    
            Flattener(),
            nn.Linear(64*2*2, 10),
          )

nn_model.type(torch.cuda.FloatTensor)
nn_model.to(device)

loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(nn_model.parameters(), lr=1e-1, weight_decay=1e-4)

Восстановите функцию `compute_accuracy` из прошлого задания.  
Единственное отличие в новом - она должна передать данные на GPU прежде чем прогонять через модель. Сделайте это так же, как это делает функция `train_model`

In [28]:
def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs, lr_scheduler=None):    
    loss_history = []
    train_history = []
    val_history = []
    for epoch in range(num_epochs):
        model.train() # Enter train mode
        
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y) in enumerate(train_loader):
          
            x_gpu = x.to(device)
            y_gpu = y.to(device)
            prediction = model(x_gpu)    
            loss_value = loss(prediction, y_gpu)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y_gpu)
            total_samples += y.shape[0]
            
            loss_accum += loss_value

        ave_loss = loss_accum / i_step
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy = compute_accuracy(model, val_loader)

        if lr_scheduler is not None:
            lr_scheduler.step()
        
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        
        print("Average loss: %f, Train accuracy: %f, Val accuracy: %f" % (ave_loss, train_accuracy, val_accuracy))
        
    return loss_history, train_history, val_history
        
def compute_accuracy(model, loader):
    """
    Computes accuracy on the dataset wrapped in a loader
    
    Returns: accuracy as a float value between 0 and 1
    """
    model.eval() # Evaluation mode
    # TODO: Copy implementation from previous assignment
    # Don't forget to move the data to device before running it through the model!
    correct_samples = 0
    total_samples = 0
    
    for i_step, (x, y) in enumerate(loader):
      x_gpu = x.to(device)
      y_gpu = y.to(device)
      prediction = model(x_gpu)
      res = torch.argmax(prediction, dim=1)
      total_samples += res.shape[0] 
      correct_samples += torch.sum(res == y_gpu)
    
    return 1. * correct_samples / total_samples

loss_history, train_history, val_history = train_model(nn_model, train_loader, val_loader, loss, optimizer, 5)

Average loss: 0.441529, Train accuracy: 0.864929, Val accuracy: 0.856733
Average loss: 0.441543, Train accuracy: 0.864929, Val accuracy: 0.856733
Average loss: 0.441444, Train accuracy: 0.864929, Val accuracy: 0.856733
Average loss: 0.441507, Train accuracy: 0.864929, Val accuracy: 0.856733
Average loss: 0.441474, Train accuracy: 0.864929, Val accuracy: 0.856733


# Аугментация данных (Data augmentation)

В работе с изображениями одним из особенно важных методов является аугментация данных - то есть, генерация дополнительных данных для тренировки на основе изначальных.   
Таким образом, мы получаем возможность "увеличить" набор данных для тренировки, что ведет к лучшей работе сети.
Важно, чтобы аугментированные данные были похожи на те, которые могут встретиться в реальной жизни, иначе польза от аугментаций уменьшается и может ухудшить работу сети.

С PyTorch идут несколько таких алгоритмов, называемых `transforms`. Более подробно про них можно прочитать тут -
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#transforms

Ниже мы используем следующие алгоритмы генерации:
- ColorJitter - случайное изменение цвета
- RandomHorizontalFlip - горизонтальное отражение с вероятностью 50%
- RandomVerticalFlip - вертикальное отражение с вероятностью 50%
- RandomRotation - случайный поворот

In [0]:
tfs = transforms.Compose([
    transforms.ColorJitter(hue=.50, saturation=.50),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(50, resample=PIL.Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.43,0.44,0.47],
                       std=[0.20,0.20,0.20])                           
])

# Create augmented train dataset
data_aug_train = dset.SVHN('./', 
                       transform=tfs
                      )

train_aug_loader = torch.utils.data.DataLoader(data_aug_train, batch_size=batch_size, 
                                           sampler=train_sampler)

Визуализируем результаты агментации (вообще, смотреть на сгенерированные данные всегда очень полезно).

In [12]:
# TODO: Visualize some augmented images!
# hint: you can create new datasets and loaders to accomplish this

# Based on the visualizations, should we keep all the augmentations?

tfs = transforms.Compose([
    transforms.ColorJitter(hue=.20, saturation=.20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10, resample=PIL.Image.BILINEAR),
])

data_aug_vis = dset.SVHN('./', 
                       transform=tfs
                      )

plt.figure(figsize=(30, 3))

for i, (x, y) in enumerate(data_aug_vis):
    if i == 10:
        break
    plt.subplot(1, 10, i+1)
    plt.grid(False)
    plt.imshow(x)
    plt.axis('off')

Все ли агментации одинаково полезны на этом наборе данных? Могут ли быть среди них те, которые собьют модель с толку?

Выберите из них только корректные

In [0]:
# TODO: 
tfs = transforms.Compose([
    transforms.ColorJitter(hue=.20, saturation=.20),
    transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.43,0.44,0.47],
                       std=[0.20,0.20,0.20])                           
])

# TODO create new instances of loaders with the augmentations you chose
data_my_aug_train = dset.SVHN('./', 
                       transform=tfs
                      )

train_aug_loader = torch.utils.data.DataLoader(data_my_aug_train, batch_size=batch_size, 
                                           sampler=train_sampler)

In [23]:
# Finally, let's train with augmentations!

# Note we shouldn't use augmentations on validation

loss_history, train_history, val_history = train_model(nn_model, train_aug_loader, val_loader, loss, optimizer, 5)

Average loss: 0.552558, Train accuracy: 0.830325, Val accuracy: 0.853935
Average loss: 0.538235, Train accuracy: 0.835597, Val accuracy: 0.837349
Average loss: 0.534926, Train accuracy: 0.836109, Val accuracy: 0.827930
Average loss: 0.522943, Train accuracy: 0.840852, Val accuracy: 0.851751
Average loss: 0.519345, Train accuracy: 0.839641, Val accuracy: 0.856733


# LeNet
Попробуем имплементировать классическую архитектуру сверточной нейронной сети, предложенную Яном ЛеКуном в 1998 году. В свое время она достигла впечатляющих результатов на MNIST, посмотрим как она справится с SVHN?
Она описана в статье ["Gradient Based Learning Applied to Document Recognition"](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf), попробуйте прочитать ключевые части и имплементировать предложенную архитетуру на PyTorch.

Реализовывать слои и функцию ошибки LeNet, которых нет в PyTorch, **не нужно** - просто возьмите их размеры и переведите в уже известные нам Convolutional, Pooling и Fully Connected layers.

Если в статье не очень понятно, можно просто погуглить LeNet и разобраться в деталях :)

In [0]:
# TODO: Implement LeNet-like architecture for SVHN task
lenet_model = nn.Sequential(
            nn.Conv2d(3, 6, 5, padding=0),
            nn.Tanh(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(6, 16, 5, padding=0),
            nn.Tanh(),
            nn.MaxPool2d(2, stride=2),    
            Flattener(),
            nn.Linear(400, 120),
            nn.Tanh(),
            nn.Linear(120, 84),
            nn.Tanh(),
            nn.Linear(84, 10)
          )

lenet_model.type(torch.cuda.FloatTensor)
lenet_model.to(device)

loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(lenet_model.parameters(), lr=1e-1, weight_decay=1e-4)

In [27]:
# Let's train it!
loss_history, train_history, val_history = train_model(lenet_model, train_aug_loader, val_loader, loss, optimizer, 10)

Average loss: 1.292604, Train accuracy: 0.563526, Val accuracy: 0.810866
Average loss: 0.655858, Train accuracy: 0.796471, Val accuracy: 0.845540
Average loss: 0.558919, Train accuracy: 0.826571, Val accuracy: 0.850317
Average loss: 0.508241, Train accuracy: 0.842815, Val accuracy: 0.863149
Average loss: 0.472683, Train accuracy: 0.852370, Val accuracy: 0.873592
Average loss: 0.448090, Train accuracy: 0.861721, Val accuracy: 0.879189
Average loss: 0.430179, Train accuracy: 0.866720, Val accuracy: 0.877210
Average loss: 0.413141, Train accuracy: 0.872846, Val accuracy: 0.882397
Average loss: 0.403406, Train accuracy: 0.875269, Val accuracy: 0.880759
Average loss: 0.392772, Train accuracy: 0.879569, Val accuracy: 0.874343


# Подбор гиперпараметров

In [32]:
# The key hyperparameters we're going to tune are learning speed, annealing rate and regularization
# We also encourage you to try different optimizers as well

Hyperparams = namedtuple("Hyperparams", ['learning_rate', 'anneal_epochs', 'reg'])
RunResult = namedtuple("RunResult", ['model', 'train_history', 'val_history', 'final_val_accuracy'])

learning_rates = [1e-1, 1e-2, 1e-3]
anneal_coeff = 0.2
anneal_epochs = [1, 5, 10, 15, 20, 50]
reg = [1e-3, 1e-4, 1e-5, 1e-7]

batch_size = 64
epoch_num = 5

# Record all the runs here
# Key should be Hyperparams and values should be RunResult
run_record = {} 

# Use grid search or random search and record all runs in run_record dictionnary 
# Important: perform search in logarithmic space!

for lr in learning_rates:
  for _ in range(3):
    an_e = np.random.choice(anneal_epochs)
    reg_str = np.random.choice(reg)

    lenet = nn.Sequential(
            nn.Conv2d(3, 6, 5, padding=0),
            nn.Tanh(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(6, 16, 5, padding=0),
            nn.Tanh(),
            nn.MaxPool2d(2, stride=2),    
            Flattener(),
            nn.Linear(400, 120),
            nn.Tanh(),
            nn.Linear(120, 84),
            nn.Tanh(),
            nn.Linear(84, 10)
          )
    
    lenet.type(torch.cuda.FloatTensor)
    lenet.to(device)

    loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
    optimizer = optim.SGD(lenet.parameters(), lr=lr, weight_decay=reg_str)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=an_e, gamma=anneal_coeff)

    loss_history, train_history, val_history = train_model(lenet, train_aug_loader, val_loader, loss, optimizer, epoch_num, lr_scheduler=scheduler)
    run_record[Hyperparams(lr, an_e, reg_str)] = RunResult(lenet, train_history, val_history, val_history[-1])

Average loss: 1.336249, Train accuracy: 0.547998, Val accuracy: 0.796737
Average loss: 0.644085, Train accuracy: 0.801983, Val accuracy: 0.850044
Average loss: 0.551137, Train accuracy: 0.830000, Val accuracy: 0.851614
Average loss: 0.504984, Train accuracy: 0.843037, Val accuracy: 0.859941
Average loss: 0.471221, Train accuracy: 0.855663, Val accuracy: 0.874275
Average loss: 1.283379, Train accuracy: 0.566495, Val accuracy: 0.823357
Average loss: 0.637107, Train accuracy: 0.802205, Val accuracy: 0.853798
Average loss: 0.549578, Train accuracy: 0.830359, Val accuracy: 0.864241
Average loss: 0.504960, Train accuracy: 0.843173, Val accuracy: 0.864173
Average loss: 0.474951, Train accuracy: 0.854298, Val accuracy: 0.877688
Average loss: 1.336412, Train accuracy: 0.550046, Val accuracy: 0.801310
Average loss: 0.645533, Train accuracy: 0.799935, Val accuracy: 0.837144
Average loss: 0.555004, Train accuracy: 0.828499, Val accuracy: 0.866357
Average loss: 0.502539, Train accuracy: 0.844214, V

In [33]:
best_val_accuracy = None
best_hyperparams = None
best_run = None

for hyperparams, run_result in run_record.items():
    if best_val_accuracy is None or best_val_accuracy < run_result.final_val_accuracy:
        best_val_accuracy = run_result.final_val_accuracy
        best_hyperparams = hyperparams
        best_run = run_result
        
print("Best validation accuracy: %4.2f, best hyperparams: %s" % (best_val_accuracy, best_hyperparams))
        

Best validation accuracy: 0.88, best hyperparams: Hyperparams(learning_rate=0.1, anneal_epochs=15, reg=1e-07)


# Свободное упражнение - догоним и перегоним LeNet!

Попробуйте найти архитектуру и настройки тренировки, чтобы выступить лучше наших бейзлайнов.

Что можно и нужно попробовать:
- BatchNormalization (для convolution layers он в PyTorch называется [batchnorm2d](https://pytorch.org/docs/stable/nn.html#batchnorm2d))
- Изменить количество слоев и их толщину
- Изменять количество эпох тренировки
- Попробовать и другие агментации

In [53]:
best_model = nn.Sequential(
            nn.Conv2d(3, 6, 5, padding=0),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(6),

            nn.Conv2d(6, 12, 5, padding=0),
            nn.MaxPool2d(2, stride=2),    
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(12),

            Flattener(),

            nn.Linear(5 * 5 * 12, 120),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(120),

            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(84),
            
            nn.Linear(84, 10)
          )

best_model.type(torch.cuda.FloatTensor)
best_model.to(device)

loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(best_model.parameters(), lr=0.1, weight_decay=1e-7)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 5, )

loss_history, train_history, val_history = train_model(best_model, train_aug_loader, val_loader, loss, optimizer, 10, lr_scheduler=scheduler)

Average loss: 0.850550, Train accuracy: 0.726547, Val accuracy: 0.848747
Average loss: 0.521791, Train accuracy: 0.836826, Val accuracy: 0.879530
Average loss: 0.453118, Train accuracy: 0.860492, Val accuracy: 0.885742
Average loss: 0.405313, Train accuracy: 0.875644, Val accuracy: 0.898505
Average loss: 0.375686, Train accuracy: 0.885217, Val accuracy: 0.901645
Average loss: 0.433829, Train accuracy: 0.866021, Val accuracy: 0.893932
Average loss: 0.406254, Train accuracy: 0.874450, Val accuracy: 0.900280
Average loss: 0.374482, Train accuracy: 0.886496, Val accuracy: 0.902874
Average loss: 0.340788, Train accuracy: 0.896546, Val accuracy: 0.908675
Average loss: 0.319712, Train accuracy: 0.901990, Val accuracy: 0.910859


# Финальный аккорд - проверим лучшую модель на test set

В качестве разнообразия - напишите код для прогона модели на test set вы.

В результате вы должны натренировать модель, которая покажет более **90%** точности на test set.  
Как водится, лучший результат в группе получит дополнительные баллы!

In [54]:
test_loader = torch.utils.data.DataLoader(data_test, batch_size=batch_size)
final_test_accuracy = compute_accuracy(best_model, test_loader)
print("Final test accuracy - ", final_test_accuracy)

Final test accuracy -  tensor(0.9038, device='cuda:0')
